In [1]:
!pip install openai --quiet
!pip install langchain --quiet
!pip install cohere --quiet
!pip install tiktoken --quiet
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00


In [2]:
import os
# os.environ['OPENAI_API_KEY'] = "Your own OPENAI_API_KEY"
# os.environ['COHERE_API_KEY'] = "Your own COHERE_API_KEY"

#Better way
from google.colab import userdata

os.environ['COHERE_API_KEY'] = userdata.get("")

In [3]:
# Helper function for printing docs


def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

# Step-1: Document Loading

In [4]:
!pip install pypdf --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.4/303.4 kB 4.8 MB/s eta 0:00:00


In [5]:
from langchain.document_loaders import PyPDFLoader

In [6]:
!wget https://raw.githubusercontent.com/venkatareddykonasani/Datasets/master/Agreements/EMPLOYEE_AGREEMENT.pdf
loader = PyPDFLoader("EMPLOYEE_AGREEMENT.pdf")
pages = loader.load()
print(len(pages))

--2025-05-27 13:36:23--  https://raw.githubusercontent.com/venkatareddykonasani/Datasets/master/Agreements/EMPLOYEE_AGREEMENT.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 206079 (201K) [application/octet-stream]
Saving to: ‘EMPLOYEE_AGREEMENT.pdf’

EMPLOYEE_AGREEMENT. 100%[===================>] 201.25K  --.-KB/s    in 0.04s   

2025-05-27 13:36:24 (5.55 MB/s) - ‘EMPLOYEE_AGREEMENT.pdf’ saved [206079/206079]

13


In [7]:
full_text =""
for page in pages:
  full_text += page.page_content

print("Pages", len(pages))
print("Lines" , len(full_text.split("\n")))
print("Words" , len(full_text.split(" ")))
print("Charecters", len(full_text))

Pages 13
Lines 369
Words 4204
Charecters 29540


# Step-2:Split the data into Chunks

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30)
chunks = text_splitter.split_documents(pages)
print(len(chunks))

133


In [10]:
print(chunks[0])

page_content='EX-10.1 3 smtp_ex10z1.htm EMPLOYEE AGREEMENT
EXHIBIT 10.1
EMPLOYEE AGREEMENT
THIS EMPLOYEE AGREEMENT made as of September___, 2014, by and between SharpSpring,
Inc., a Delaware corporation (the “Company”), whose principal place of business is at 802 NW 5th Avenue,' metadata={'producer': 'Skia/PDF m123', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36', 'creationdate': '2024-04-03T12:51:00+00:00', 'title': 'EMPLOYEE AGREEMENT', 'moddate': '2024-04-03T12:51:00+00:00', 'source': 'EMPLOYEE_AGREEMENT.pdf', 'total_pages': 13, 'page': 0, 'page_label': '1'}


# Step-3: Creating embeddings

In [11]:
from langchain.embeddings import OpenAIEmbeddings, CohereEmbeddings

In [12]:
#embeddings = OpenAIEmbeddings()
embeddings = CohereEmbeddings(user_agent="langchain")

<ipython-input-12-7e421fe0b3eb>:2: LangChainDeprecationWarning: The class `CohereEmbeddings` was deprecated in LangChain 0.0.30 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-cohere package and should be used instead. To use it run `pip install -U :class:`~langchain-cohere` and import as `from :class:`~langchain_cohere import CohereEmbeddings``.
  embeddings = CohereEmbeddings(user_agent="langchain")


In [13]:
# A sample embedding

sample_embedding = embeddings.embed_query("You must follow the rules")
print(sample_embedding)

[0.9038086, 0.22033691, -1.1132812, 0.87109375, 0.7758789, -0.8671875, -1.7265625, -3.4882812, -0.79785156, -0.9248047, 0.7158203, 0.12878418, -2.1992188, -1.4003906, 1.71875, -2.2148438, 1.2626953, 3.6875, 1.5859375, -0.8857422, 0.65478516, 0.81152344, 0.47265625, 0.39990234, 3.5410156, -0.4165039, 2.3496094, -2.7578125, -1.3652344, 0.9609375, 0.051849365, 0.72753906, 0.89208984, 1.8652344, -1.4140625, -1.7783203, 2.2910156, -0.8955078, -0.10760498, 1.2958984, -2.4003906, 1.0673828, 0.12963867, -1.7050781, 2.0976562, -1.7285156, -1.3632812, 3.1933594, 1.6357422, -0.96875, 2.4628906, 0.27783203, -0.42333984, 0.7553711, -0.7441406, -1.8066406, -3.5625, -0.84472656, -0.5419922, -0.30029297, -0.28881836, -0.79052734, 1.1015625, -1.3984375, 0.3618164, -1.7236328, -1.6503906, -1.9824219, 0.1928711, 0.72753906, 0.18774414, 1.8144531, 2.2539062, -0.56347656, 0.27514648, 1.2304688, -1.4589844, 1.5078125, 0.4831543, -1.9550781, -0.82128906, 0.85302734, -1.2285156, 1.2070312, -1.0117188, -1.3183

In [14]:
len(sample_embedding)

4096

In [15]:
sample_docs=["You must follow the rules",
            "You must not disclose the rules"]

embeded_vectors1= embeddings.embed_documents(sample_docs)
print(embeded_vectors1)

[[0.9038086, 0.22033691, -1.1132812, 0.87109375, 0.7758789, -0.8671875, -1.7265625, -3.4882812, -0.79785156, -0.9248047, 0.7158203, 0.12878418, -2.1992188, -1.4003906, 1.71875, -2.2148438, 1.2626953, 3.6875, 1.5859375, -0.8857422, 0.65478516, 0.81152344, 0.47265625, 0.39990234, 3.5410156, -0.4165039, 2.3496094, -2.7578125, -1.3652344, 0.9609375, 0.051849365, 0.72753906, 0.89208984, 1.8652344, -1.4140625, -1.7783203, 2.2910156, -0.8955078, -0.10760498, 1.2958984, -2.4003906, 1.0673828, 0.12963867, -1.7050781, 2.0976562, -1.7285156, -1.3632812, 3.1933594, 1.6357422, -0.96875, 2.4628906, 0.27783203, -0.42333984, 0.7553711, -0.7441406, -1.8066406, -3.5625, -0.84472656, -0.5419922, -0.30029297, -0.28881836, -0.79052734, 1.1015625, -1.3984375, 0.3618164, -1.7236328, -1.6503906, -1.9824219, 0.1928711, 0.72753906, 0.18774414, 1.8144531, 2.2539062, -0.56347656, 0.27514648, 1.2304688, -1.4589844, 1.5078125, 0.4831543, -1.9550781, -0.82128906, 0.85302734, -1.2285156, 1.2070312, -1.0117188, -1.318

In [16]:
print(len(embeded_vectors1))
print(len(embeded_vectors1[0]))

2
4096


In [17]:
print(sample_embedding)
print(embeded_vectors1[0])

[0.9038086, 0.22033691, -1.1132812, 0.87109375, 0.7758789, -0.8671875, -1.7265625, -3.4882812, -0.79785156, -0.9248047, 0.7158203, 0.12878418, -2.1992188, -1.4003906, 1.71875, -2.2148438, 1.2626953, 3.6875, 1.5859375, -0.8857422, 0.65478516, 0.81152344, 0.47265625, 0.39990234, 3.5410156, -0.4165039, 2.3496094, -2.7578125, -1.3652344, 0.9609375, 0.051849365, 0.72753906, 0.89208984, 1.8652344, -1.4140625, -1.7783203, 2.2910156, -0.8955078, -0.10760498, 1.2958984, -2.4003906, 1.0673828, 0.12963867, -1.7050781, 2.0976562, -1.7285156, -1.3632812, 3.1933594, 1.6357422, -0.96875, 2.4628906, 0.27783203, -0.42333984, 0.7553711, -0.7441406, -1.8066406, -3.5625, -0.84472656, -0.5419922, -0.30029297, -0.28881836, -0.79052734, 1.1015625, -1.3984375, 0.3618164, -1.7236328, -1.6503906, -1.9824219, 0.1928711, 0.72753906, 0.18774414, 1.8144531, 2.2539062, -0.56347656, 0.27514648, 1.2304688, -1.4589844, 1.5078125, 0.4831543, -1.9550781, -0.82128906, 0.85302734, -1.2285156, 1.2070312, -1.0117188, -1.3183

# Step-4: Storing in Vector Stores

In [18]:
!pip install chromadb -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.0/19.0 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 8.0 MB/s eta 0:

In [19]:
from langchain.vectorstores import Chroma

In [20]:
emp_rules_db= Chroma.from_documents(chunks,
                                    embeddings,
                                    persist_directory="emp_rules_db"
                          )
emp_rules_db.persist()

<ipython-input-20-b50aff525da8>:5: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  emp_rules_db.persist()


# Step-5: Retrieval

In [21]:
retriever = emp_rules_db.as_retriever(search_kwargs={"k": 3})
result = retriever.invoke("What is the policy for sick leaves")
print(result)


[Document(metadata={'total_pages': 13, 'source': 'EMPLOYEE_AGREEMENT.pdf', 'producer': 'Skia/PDF m123', 'creationdate': '2024-04-03T12:51:00+00:00', 'moddate': '2024-04-03T12:51:00+00:00', 'page': 1, 'page_label': '2', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36', 'title': 'EMPLOYEE AGREEMENT'}, page_content='sick days and personal absence days as authorized by the Company for its other employees.\nARTICLE 6\nFringe Benefits\nEmployee shall be entitled to participate in and receive benefits from all of the Company’s employee\nbenefit plans that are now, or in the future may be, maintained by the'), Document(metadata={'total_pages': 13, 'moddate': '2024-04-03T12:51:00+00:00', 'page': 1, 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36', 'creationdate': '2024-04-03T12:51:00+00:00', 'source': 'EMPLOYEE_AGREEMENT.pdf', 'title': 'EMPLOYEE

In [22]:
for i in range(len(result)):
  print(result[i].metadata)

{'total_pages': 13, 'source': 'EMPLOYEE_AGREEMENT.pdf', 'producer': 'Skia/PDF m123', 'creationdate': '2024-04-03T12:51:00+00:00', 'moddate': '2024-04-03T12:51:00+00:00', 'page': 1, 'page_label': '2', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36', 'title': 'EMPLOYEE AGREEMENT'}
{'total_pages': 13, 'moddate': '2024-04-03T12:51:00+00:00', 'page': 1, 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36', 'creationdate': '2024-04-03T12:51:00+00:00', 'source': 'EMPLOYEE_AGREEMENT.pdf', 'title': 'EMPLOYEE AGREEMENT', 'producer': 'Skia/PDF m123', 'page_label': '2'}
{'title': 'EMPLOYEE AGREEMENT', 'page_label': '2', 'source': 'EMPLOYEE_AGREEMENT.pdf', 'creationdate': '2024-04-03T12:51:00+00:00', 'producer': 'Skia/PDF m123', 'total_pages': 13, 'page': 1, 'moddate': '2024-04-03T12:51:00+00:00', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) A

You can also set a retrieval method that sets a similarity score threshold and only returns documents with a score above that threshold.


### pretty_print_docs

In [23]:
retriever = emp_rules_db.as_retriever(search_kwargs={"k": 3})
result = retriever.invoke("What is the policy for insurance?")
print(result)



[Document(metadata={'moddate': '2024-04-03T12:51:00+00:00', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36', 'producer': 'Skia/PDF m123', 'source': 'EMPLOYEE_AGREEMENT.pdf', 'page_label': '3', 'creationdate': '2024-04-03T12:51:00+00:00', 'total_pages': 13, 'title': 'EMPLOYEE AGREEMENT', 'page': 2}, page_content='Company for its employees, including, without limitation, the Company’s health insurance plan. No amounts\npaid to Employee from an employee benefit plan shall count as compensation due Employee as base salary or'), Document(metadata={'page_label': '7', 'producer': 'Skia/PDF m123', 'title': 'EMPLOYEE AGREEMENT', 'source': 'EMPLOYEE_AGREEMENT.pdf', 'page': 6, 'creationdate': '2024-04-03T12:51:00+00:00', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36', 'moddate': '2024-04-03T12:51:00+00:00', 'total_pages': 13}, page_content='sha

In [24]:
for i in range(len(result)):
  print(result[i].metadata)

{'moddate': '2024-04-03T12:51:00+00:00', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36', 'producer': 'Skia/PDF m123', 'source': 'EMPLOYEE_AGREEMENT.pdf', 'page_label': '3', 'creationdate': '2024-04-03T12:51:00+00:00', 'total_pages': 13, 'title': 'EMPLOYEE AGREEMENT', 'page': 2}
{'page_label': '7', 'producer': 'Skia/PDF m123', 'title': 'EMPLOYEE AGREEMENT', 'source': 'EMPLOYEE_AGREEMENT.pdf', 'page': 6, 'creationdate': '2024-04-03T12:51:00+00:00', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36', 'moddate': '2024-04-03T12:51:00+00:00', 'total_pages': 13}
{'title': 'EMPLOYEE AGREEMENT', 'total_pages': 13, 'page': 6, 'page_label': '7', 'producer': 'Skia/PDF m123', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36', 'source': 'EMPLOYEE_AGREEMENT.pdf', 'creationdate'

In [25]:
retriever = emp_rules_db.as_retriever(search_kwargs={"k": 3})
result = retriever.invoke("Employee Monthly Salary?")
print(result)


[Document(metadata={'page_label': '2', 'producer': 'Skia/PDF m123', 'page': 1, 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36', 'total_pages': 13, 'source': 'EMPLOYEE_AGREEMENT.pdf', 'moddate': '2024-04-03T12:51:00+00:00', 'creationdate': '2024-04-03T12:51:00+00:00', 'title': 'EMPLOYEE AGREEMENT'}, page_content='Compensation of Employee\n4.1. Base Compensation.  For all services rendered by Employee under this Employee Agreement, the\nCompany agrees to pay Employee the rate of $14,583 per month (the “base salary”), which shall be payable'), Document(metadata={'title': 'EMPLOYEE AGREEMENT', 'moddate': '2024-04-03T12:51:00+00:00', 'producer': 'Skia/PDF m123', 'source': 'EMPLOYEE_AGREEMENT.pdf', 'page_label': '2', 'total_pages': 13, 'creationdate': '2024-04-03T12:51:00+00:00', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36', 'page': 1}, 

In [26]:
for i in range(len(result)):
  print(result[i].metadata)

{'page_label': '2', 'producer': 'Skia/PDF m123', 'page': 1, 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36', 'total_pages': 13, 'source': 'EMPLOYEE_AGREEMENT.pdf', 'moddate': '2024-04-03T12:51:00+00:00', 'creationdate': '2024-04-03T12:51:00+00:00', 'title': 'EMPLOYEE AGREEMENT'}
{'title': 'EMPLOYEE AGREEMENT', 'moddate': '2024-04-03T12:51:00+00:00', 'producer': 'Skia/PDF m123', 'source': 'EMPLOYEE_AGREEMENT.pdf', 'page_label': '2', 'total_pages': 13, 'creationdate': '2024-04-03T12:51:00+00:00', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36', 'page': 1}
{'page_label': '3', 'total_pages': 13, 'producer': 'Skia/PDF m123', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36', 'creationdate': '2024-04-03T12:51:00+00:00', 'moddate': '2024-04-03T12:51:00+00:00', 'title'

# RAG Example-2

In [27]:
# Step-1:Document Loading
from langchain.document_loaders import PyPDFLoader
!wget https://raw.githubusercontent.com/venkatareddykonasani/Datasets/master/Banking_System_Doc/BASEL.pdf
loader = PyPDFLoader("BASEL.pdf")
pages = loader.load()
print(len(pages))

--2025-05-27 13:39:44--  https://raw.githubusercontent.com/venkatareddykonasani/Datasets/master/Banking_System_Doc/BASEL.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1260197 (1.2M) [application/octet-stream]
Saving to: ‘BASEL.pdf’

BASEL.pdf           100%[===================>]   1.20M  --.-KB/s    in 0.06s   

2025-05-27 13:39:44 (20.0 MB/s) - ‘BASEL.pdf’ saved [1260197/1260197]

77


In [28]:
#Step-2:Split the data into Chunks

from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(pages)
print(len(chunks))

521


In [29]:
#Step-3: Creating Embeddings
from langchain.embeddings import OpenAIEmbeddings, CohereEmbeddings
#embeddings = OpenAIEmbeddings()
embeddings = CohereEmbeddings(user_agent="langchain")

#Step-4: Storing in a Vector DB
#!rm -rf basel_norms_db
basel_norms_db= Chroma.from_documents(chunks,
                                    embeddings,
                                    persist_directory="basel_norms_db"
                          )
basel_norms_db.persist()

In [30]:
# Step-5 Retrieval
retriever = basel_norms_db.as_retriever(search_kwargs={"k": 3})
result = retriever.invoke("What percentage is the minimum Capital Requirements")

print(result)
print(len(result))
print([i.metadata for i in result])


[Document(metadata={'author': 'Basel Committee on Banking Supervision', 'contenttype': 'Document', 'comments': '', 'keywords': '', 'description0': '', 'agenda item1': '', 'doc #': '', 'source': 'BASEL.pdf', 'subject': 'Full text of "Basel III: A global regulatory framework for more resilient banks and banking systems - post BCBS meeting - revised version", June 2011', 'moddate': '2019-11-14T17:33:29+01:00', 'meeting': '2010 - 138th Meeting of the BCBS November-December 2010', 'company': '', 'page_label': '68', 'total_pages': 77, 'sort order': '0.110000000000000', 'creationdate': '2011-05-31T11:52:06+02:00', 'creator': 'Acrobat PDFMaker 9.1 for Word', 'title': 'Basel III: A global regulatory framework for more resilient banks and banking systems - post BCBS meeting - revised version June 2011', 'page': 67, 'producer': 'Acrobat Distiller 9.4.0 (Windows)'}, page_content='capital) \nMinimum Capital Conservation Ratios \n(expressed as a percentage of earnings) \n4.5% -  5.75% 100% \n>5.75% 

In [31]:
retriever = basel_norms_db.as_retriever(search_kwargs={"k": 3})
result = retriever.invoke("What are PD and LGD")

print(result)
print(len(result))
print([i.metadata for i in result])


[Document(metadata={'meeting': '2010 - 138th Meeting of the BCBS November-December 2010', 'moddate': '2019-11-14T17:33:29+01:00', 'page': 6, 'company': '', 'creator': 'Acrobat PDFMaker 9.1 for Word', 'author': 'Basel Committee on Banking Supervision', 'sort order': '0.110000000000000', 'subject': 'Full text of "Basel III: A global regulatory framework for more resilient banks and banking systems - post BCBS meeting - revised version", June 2011', 'comments': '', 'total_pages': 77, 'creationdate': '2011-05-31T11:52:06+02:00', 'source': 'BASEL.pdf', 'producer': 'Acrobat Distiller 9.4.0 (Windows)', 'contenttype': 'Document', 'description0': '', 'page_label': '7', 'title': 'Basel III: A global regulatory framework for more resilient banks and banking systems - post BCBS meeting - revised version June 2011', 'agenda item1': '', 'keywords': '', 'doc #': ''}, page_content='LCR  Liquidity Coverage Ratio \nLGD  Loss given default \nMtM  Mark-to-market \nNSFR  Net Stable Funding Ratio \nOBS  Off

# MultiQueryRetriever

In [32]:
!pip install wikipedia -q

  Preparing metadata (setup.py) ... done


In [33]:
from langchain.document_loaders import wikipedia

#Document Loading
loader=wikipedia.WikipediaLoader(query="MS Dhoni")
documents=loader.load()

#Splitting
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)
docs=text_splitter.split_documents(documents)
len(docs)

#Embeddings and VectorDB
from langchain.embeddings import OpenAIEmbeddings
#embeddings=OpenAIEmbeddings()
embeddings = CohereEmbeddings(user_agent="langchain")

embeddings_db=Chroma.from_documents(docs,embeddings,
                                    persist_directory="wiki_db")
embeddings_db.persist()



In [34]:
from langchain.retrievers import MultiQueryRetriever
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI , Cohere


llm=Cohere(temperature=0)

<ipython-input-34-77f61949807c>:6: LangChainDeprecationWarning: The class `Cohere` was deprecated in LangChain 0.1.14 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-cohere package and should be used instead. To use it run `pip install -U :class:`~langchain-cohere` and import as `from :class:`~langchain_cohere import Cohere``.
  llm=Cohere(temperature=0)


In [35]:
llm_based_retriver=MultiQueryRetriever.from_llm(
    retriever=embeddings_db.as_retriever(),
    llm=llm
)

In [36]:
llm_based_retriver

MultiQueryRetriever(retriever=VectorStoreRetriever(tags=['Chroma', 'CohereEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7df5ac7d4950>, search_kwargs={}), llm_chain=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='You are an AI language model assistant. Your task is \n    to generate 3 different versions of the given user \n    question to retrieve relevant documents from a vector  database. \n    By generating multiple perspectives on the user question, \n    your goal is to help the user overcome some of the limitations \n    of distance-based similarity search. Provide these alternative \n    questions separated by newlines. Original question: {question}')
| Cohere(client=<cohere.client.Client object at 0x7df5ac86a9d0>, async_client=<cohere.client.AsyncClient object at 0x7df5ac872e10>, temperature=0.0, cohere_api_key=SecretStr('**********'))
| LineListOutputParser())

In [37]:
import logging
logging.basicConfig(level=logging.INFO)
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [38]:
question1="What is the DOB of Dhoni?"
question2= "What Sport does Dhoni Play?"
rel_docs1=llm_based_retriver.get_relevant_documents(question1)
rel_docs2=llm_based_retriver.get_relevant_documents(question2)

<ipython-input-38-3089a50f7a57>:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  rel_docs1=llm_based_retriver.get_relevant_documents(question1)
INFO:langchain.retrievers.multi_query:Generated queries: ['Here are 3 different versions of the original question, to help with gathering relevant information from the database:', '1. DOB refers to Date of Birth and Dhoni is a former Indian cricket team captain -- do you have information on cricketers in your database?', '2. I am looking for the date of birth of individuals named Dhoni, do you have any records or metadata that can help?', '3. Search for the birth date of the Indian national cricket team captain named Dhoni. ', 'These variations help provide some context and specify the search query to get more targeted results from the vector database, while overcoming limitations of simple keyword or exact match s

In [39]:
rel_docs1

[Document(metadata={'title': 'List of Indian Premier League records and statistics', 'source': 'https://en.wikipedia.org/wiki/List_of_Indian_Premier_League_records_and_statistics', 'summary': "The Indian Premier League, popularly knows as IPL, is a men's T20 cricket league, organized by the Board of Control for Cricket in India, the national governing body of cricket in India. T20 cricket is a shorter format of the game, with each team playing a single innings of 20 overs. The  inaugural season of the IPL was held in 2008, and the tournament has been held annually in India ever since. Currently, ten teams compete for the championship. Since the league's inception, seven teams have won the title, with the Mumbai Indians and Chennai Super Kings both securing five titles each.\nIndian batsman Virat Kohli holds the records for the most runs scored (8,552), the most centuries (8), most half-centuries (62), most runs in a single season (973), most runs as a captain (4,994) and the most catch

In [40]:
rel_docs2

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/MS_Dhoni', 'title': 'MS Dhoni', 'summary': "Mahendra Singh Dhoni ( ; born 7 July 1981) is an Indian professional cricketer who plays as a right-handed batter and a wicket-keeper. Widely regarded as one of the most prolific wicket-keeper batsmen and captains and one of the greatest ODI batsmen, he represented the Indian cricket team and was the captain of the side in limited overs formats from 2007 to 2017 and in test cricket from 2008 to 2014. Dhoni has captained the most international matches and is the most successful Indian captain. He has led India to victory in the 2007 ICC World Twenty20, the 2011 Cricket World Cup, and the 2013 ICC Champions Trophy, being the only captain to win three different limited overs ICC tournaments. He also led the teams that won the Asia Cup in 2010, 2016 and was a member of the title winning squad in 2018.\nBorn in Ranchi, Dhoni made his first class debut for Bihar in 1999. He made his debut

# Contextual compression

In [41]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [42]:
#llm=OpenAI(temperature=0)
llm=Cohere(temperature=0)

compressor=LLMChainExtractor.from_llm(llm)

compression_retriever=ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=embeddings_db.as_retriever()
)

In [43]:
compressed_docs=compression_retriever.get_relevant_documents(question1)

In [44]:
compressed_docs[0].metadata

{'summary': "Mahendra Singh Dhoni ( ; born 7 July 1981) is an Indian professional cricketer who plays as a right-handed batter and a wicket-keeper. Widely regarded as one of the most prolific wicket-keeper batsmen and captains and one of the greatest ODI batsmen, he represented the Indian cricket team and was the captain of the side in limited overs formats from 2007 to 2017 and in test cricket from 2008 to 2014. Dhoni has captained the most international matches and is the most successful Indian captain. He has led India to victory in the 2007 ICC World Twenty20, the 2011 Cricket World Cup, and the 2013 ICC Champions Trophy, being the only captain to win three different limited overs ICC tournaments. He also led the teams that won the Asia Cup in 2010, 2016 and was a member of the title winning squad in 2018.\nBorn in Ranchi, Dhoni made his first class debut for Bihar in 1999. He made his debut for the Indian cricket team on 23 December 2004 in an ODI against Bangladesh and played his

In [45]:
compressed_docs[0].metadata["summary"]

"Mahendra Singh Dhoni ( ; born 7 July 1981) is an Indian professional cricketer who plays as a right-handed batter and a wicket-keeper. Widely regarded as one of the most prolific wicket-keeper batsmen and captains and one of the greatest ODI batsmen, he represented the Indian cricket team and was the captain of the side in limited overs formats from 2007 to 2017 and in test cricket from 2008 to 2014. Dhoni has captained the most international matches and is the most successful Indian captain. He has led India to victory in the 2007 ICC World Twenty20, the 2011 Cricket World Cup, and the 2013 ICC Champions Trophy, being the only captain to win three different limited overs ICC tournaments. He also led the teams that won the Asia Cup in 2010, 2016 and was a member of the title winning squad in 2018.\nBorn in Ranchi, Dhoni made his first class debut for Bihar in 1999. He made his debut for the Indian cricket team on 23 December 2004 in an ODI against Bangladesh and played his first test 

# RetrievalQA Chain

In [46]:
question1="What is the DOB of Dhoni?"
question2= "What Sport does Dhoni Play?"
rel_docs1=llm_based_retriver.get_relevant_documents(question1)
rel_docs2=llm_based_retriver.get_relevant_documents(question2)

INFO:langchain.retrievers.multi_query:Generated queries: ['Here are 3 different versions of the original question, to help with gathering relevant information from the database:', '1. DOB refers to Date of Birth and Dhoni is a former Indian cricket team captain -- do you have information on cricketers in your database?', '2. I am looking for the date of birth of individuals named Dhoni, do you have any records or metadata that can help?', '3. Search for the birth date of the Indian national cricket team captain named Dhoni. ', 'These variations help provide some context and specify the search query to get more targeted results from the vector database, while overcoming limitations of simple keyword or exact match searches.']
INFO:langchain.retrievers.multi_query:Generated queries: ['1. What sports is M.S. Dhoni involved in? ', '2. List some sports that are popular with celebrities like M.S. Dhoni. ', '3. What sports legends like M.S. Dhoni participate in?']


In [47]:
from langchain.chains import RetrievalQA
#llm=OpenAI(temperature=0)
llm=Cohere(temperature=0)

Q_AChain=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",# It takes a list of documents, inserts them all into a prompt
    retriever=llm_based_retriver
)

In [48]:
query="What is the DOB of Dhoni?"
docs=Q_AChain({"query":query})
docs["result"]

<ipython-input-48-6ae75a3c7e7c>:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs=Q_AChain({"query":query})
INFO:langchain.retrievers.multi_query:Generated queries: ['Here are 3 different versions of the original question, to help with gathering relevant information from the database:', '1. DOB refers to Date of Birth and Dhoni is a former Indian cricket team captain -- do you have information on cricketers in your database?', '2. I am looking for the date of birth of individuals named Dhoni, do you have any records or meta-data that could help?', "3. Search vector database for records pertaining to biodata of Indian cricket team captains, where the name includes the term 'Dhoni'. ", 'These variations help overcome some of the limitations of distance-based similarity search in vector databases by providing more context and clarity around the query. Let me know if you would like m

' Dhoni was born on July 7, 1981 and started his career in 1995, playing for Commando cricket club. '

In [49]:
docs

{'query': 'What is the DOB of Dhoni?',
 'result': ' Dhoni was born on July 7, 1981 and started his career in 1995, playing for Commando cricket club. '}

In [50]:
print(Q_AChain.combine_documents_chain.llm_chain.prompt.template)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:


#Validation of the Results

In [52]:
# 1. Install Required Packages
!pip install -q langchain langchain-community cohere chromadb tiktoken

# 2. Download the COI.pdf File
!wget -q https://raw.githubusercontent.com/venkatareddykonasani/Datasets/master/COI/COI.pdf

# 3. Load PDF Using LangChain
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("COI.pdf")
pages = loader.load()

# 4. Combine Text
full_text = ""
for page in pages:
    full_text += page.page_content

print("Pages:", len(pages))
print("Lines:", len(full_text.split("\n")))
print("Words:", len(full_text.split(" ")))
print("Characters:", len(full_text))



Pages: 402
Lines: 14456
Words: 163494
Characters: 860453


In [53]:
# 5. Split into Chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=400)
chunks = text_splitter.split_documents(pages)
print("Chunks:", len(chunks))



Chunks: 681


In [56]:
# 6. Use Cohere for Embeddings (NO OpenAI)
from langchain_community.embeddings import CohereEmbeddings
from langchain.vectorstores import Chroma
import getpass

# Enter your Cohere API key securely
cohere_api_key = getpass.getpass("Enter you cohere api key")

# Initialize Cohere Embeddings
embeddings = CohereEmbeddings(cohere_api_key=cohere_api_key,user_agent="langchain")


Enter you cohere api key··········


In [ ]:
# 7. Create Vector DB
coi_db = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory="coi_db"
)
coi_db.persist()

In [ ]:
# 8. Retrieval QA Chain Using Cohere as LLM
from langchain.chains import RetrievalQA
from langchain_community.llms import Cohere as CohereLLM  # Use Cohere as LLM

llm = CohereLLM(cohere_api_key=cohere_api_key, temperature=0.3)

In [ ]:
# 9. Build the Retrieval QA Chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=coi_db.as_retriever()
)

# 10. Ask a Sample Question
response = qa_chain.run("What is the main objective of the COI document?")
print("\nResponse:\n", response)

In [ ]:
query="""
According to constitution of India what are the fundamental rights of citizens of India ?
"""
docs=Q_AChain({"query":query})
print(docs["result"])

## RAG Validation Process

In [ ]:
llm=OpenAI(temperature=0)

Q_AChain=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",# It takes a list of documents, inserts them all into a prompt
    retriever=coi_db.as_retriever(),
    input_key="question" # Add this extra parameter to access the user questions using input_key
)

##Get Validation data

In [ ]:
!wget https://raw.githubusercontent.com/venkatareddykonasani/Datasets/master/COI/COI_Q_A.csv

In [ ]:
import pandas as pd
test_data=pd.read_csv("COI_Q_A.csv")
test_data.head()

#Creating a iterative "Question", "Answer" pairs
test_qa_pairs = []
for index, row in test_data.iterrows():
  question = row['Question']
  answer = row['Answer']
  test_qa_pairs.append({'question': question, 'answer': answer})
print(test_qa_pairs)

In [ ]:
#Add one more Question and Answer
question_new="Who is Narendra Modi?"
answer_new="""
Narendra Modi is the current Prime Minister of India, serving since 2014.
He is a member of the Bharatiya Janata Party (BJP) and
previously served as the Chief Minister of Gujarat from 2001 to 2014.
"""
test_qa_pairs.append({'question': question_new, 'answer': answer_new})

## Get the Predictions

In [ ]:
predictions = Q_AChain.apply(test_qa_pairs)
predictions

## Compare Actual Answers with predicted Answers

In [ ]:
#The comparison is done using an LLM
from langchain.evaluation.qa import QAEvalChain

llm=OpenAI(temperature=0)

qa_eval_chain = QAEvalChain.from_llm(llm)

In [ ]:
eval_result = qa_eval_chain.evaluate(test_qa_pairs,
                                     predictions,
                                      question_key="question",
                                      answer_key="answer"
                                     )
print(eval_result)

## Calculate The Accuracy

In [ ]:
# Initialize counters
correct_count = 0
incorrect_count = 0

# Count occurrences
for item in eval_result:
    if item['results'].strip() == 'CORRECT':
        correct_count += 1
    elif item['results'].strip() == 'INCORRECT':
        incorrect_count += 1

# Calculate percentages
total_count = len(eval_result)
correct_percentage = (correct_count / total_count) * 100
incorrect_percentage = (incorrect_count / total_count) * 100

# Print the results
print(f"CORRECT: {correct_percentage}%")
print(f"INCORRECT: {incorrect_percentage}%")
